In [2]:
import requests
#import pandas as pd

In [8]:
def transform_data(json_file: list):
    '''
    Function used to trasnform a json file into a new dictionary containing all relevant pokemon data to our work

    json_file: list containing the json response file from the API
    '''

    data = []
    for pokemon in json_file:
        
        # Stores relevant data for each pokemon in a dict
        pokemon_data = dict()
        

        # Get name
        pokemon_data['name'] = pokemon.get('names', {}).get('English', None)
        # Get stats safely
        stats = pokemon.get('stats', {})
        if stats:  # Check if stats is not None
            pokemon_data['stamina'] = stats.get('stamina', None)
            pokemon_data['attack'] = stats.get('attack', None)
            pokemon_data['defense'] = stats.get('defense', None)
        else:
            pokemon_data['stamina'] = None
            pokemon_data['attack'] = None
            pokemon_data['defense'] = None

        pokemon_data['primaryType'] = pokemon.get('primaryType').get('names').get('English')
        
        try:
            pokemon_data['secondaryType'] = pokemon.get('secondaryType').get('names').get('English')
        except AttributeError:
            pokemon_data['secondaryType'] = None

    

    
    


try:
    r = requests.get('https://pokemon-go-api.github.io/pokemon-go-api/api/pokedex.json')
    r.raise_for_status()
except requests.exceptions.RequestException as e:
    print('Erro:', e)  

# Get content of the response
r_content = r.json()






In [44]:
pokemon['quickMoves']

{'THUNDER_SHOCK_FAST': {'id': 'THUNDER_SHOCK_FAST',
  'power': 5,
  'energy': 8,
  'durationMs': 600,
  'type': {'type': 'POKEMON_TYPE_ELECTRIC',
   'names': {'English': 'Electric',
    'German': 'Elektro',
    'French': 'Électrik',
    'Italian': 'Elettro',
    'Japanese': 'でんき',
    'Korean': '전기',
    'Spanish': 'Eléctrico'}},
  'names': {'English': 'Thunder Shock',
   'German': 'Donnerschock',
   'French': 'Éclair',
   'Italian': 'Tuonoshock',
   'Japanese': 'でんきショック',
   'Korean': '전기쇼크',
   'Spanish': 'Impactrueno'},
  'combat': {'energy': 9, 'power': 3, 'turns': 2, 'buffs': None}},
 'DRAGON_BREATH_FAST': {'id': 'DRAGON_BREATH_FAST',
  'power': 6,
  'energy': 4,
  'durationMs': 500,
  'type': {'type': 'POKEMON_TYPE_DRAGON',
   'names': {'English': 'Dragon',
    'German': 'Drache',
    'French': 'Dragon',
    'Italian': 'Drago',
    'Japanese': 'ドラゴン',
    'Korean': '드래곤',
    'Spanish': 'Dragón'}},
  'names': {'English': 'Dragon Breath',
   'German': 'Feuerodem',
   'French': 'Dr

In [42]:
data = []

for pokemon in r_content:
    
    # Stores relevant data for each pokemon in a dict
    pokemon_data = dict()
    

    # Get name
    pokemon_data['name'] = pokemon.get('names', {}).get('English', None)
    # Get stats safely
    stats = pokemon.get('stats', {})
    if stats:  # Check if stats is not None
        pokemon_data['stamina'] = stats.get('stamina', None)
        pokemon_data['attack'] = stats.get('attack', None)
        pokemon_data['defense'] = stats.get('defense', None)
    else:
        pokemon_data['stamina'] = None
        pokemon_data['attack'] = None
        pokemon_data['defense'] = None

    pokemon_data['primaryType'] = pokemon.get('primaryType').get('names').get('English')
    
    try:
        pokemon_data['secondaryType'] = pokemon.get('secondaryType').get('names').get('English')
    except AttributeError:
        pokemon_data['secondaryType'] = None





In [43]:
pokemon_data

{'name': 'Miraidon',
 'stamina': 205,
 'attack': 263,
 'defense': 223,
 'primaryType': 'Electric',
 'secondaryType': 'Dragon'}

In [13]:
r_content[0]['stats']['stamina']

128